# Implementar un Multilayer Perceptron  con PyTorch para clasificación de texto
# Uso de word embeddings como representación de datos


<img src="figs/fig-diagrama-clasificador2.png" width="900">


### 1. **Representar los datos en el modelo de word embeddings seleccionado**:  
   - #### Generalmente, solo se tokeniza para separar adecuadamente las palabras.
   - #### Sin embargo, dependiendo del modelos de word embeddings algunos preprocesamientos puede mejorar la representación.
   - #### Por ejemplo: 
      - ##### tokenizar y separar correctamente las oraciones y palabras
      - ##### convertir a minúsculas
      - ##### quitar acentos (dependiendo de la fuente de datos con la que se generaros los embeddings)
      - ##### quitar números y puntuación 


### 3. **Convertir los datos a vectores densos: word embeddings**:  
   - #### En el caso de textos corto a nivel de oración, un vector denso por oración. 

### 4. **Separar los datos para entrenamiento, validación y prueba**:  
   - #### Crear los dataset  con la función train_test_split 
   
### 5. **Definir la arquitectura de la red**:  
   - Definir una red de 2 capas, con funciones PReLU en las capas ocultas y una capa de salida

### 6. **Entrenar el modelo**:  
   - Definir los parámetros de las red como: número de épocas, learning_rate, número de neuronas para las capas ocultas, etc.
   
### 7. **Evaluar el modelo**:  
   - Después del entrenamiento, probar la red con las entradas del conjunto de test y evaluar el desempeño con las métricas: Precisión, Recall, F1-score o F1-Measure y Accuracy.
   

### 8. **Guardar el modelo**:  
   - Después del entrenamiento, el modelo se guarda para posteriores usos: despliegue del modelo para aplicación web, o como punto de partida para optimizar por medio de Computación Evolutiva.
   

# Cargar los datos de entrenamiento

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
import fasttext

# colocar la semilla para la generación de números aleatorios para la reproducibilidad de experimentos

random_state = 42
torch.manual_seed(random_state)
np.random.seed(random_state)

#cargar los datos
dataset_train = pd.read_json("./data/dataset_polaridad_es_train_embeddings.json", lines=True)
dataset_test = pd.read_json("./data/dataset_polaridad_es_test_embeddings.json", lines=True)

#conteo de clases
print("Total de ejemplos de entrenamiento")
print(dataset_train.klass.value_counts())
# Extracción de los vectores de embeddings que representan a los textos
X = np.vstack(dataset_train['we_es'].to_numpy())
# Extracción de las etiquetas o clases de entrenamiento
Y = dataset_train['klass'].to_numpy()

X_test = np.vstack(dataset_test['we_es'].to_numpy())
# Extracción de las etiquetas o clases de entrenamiento
Y_test = dataset_test['klass'].to_numpy()


print("Datos train:", X.shape) 
print("Etiquetas train:", Y.shape) 
print("Datos test:", X_test.shape) 
print("Etiquetas test:", Y_test.shape) 



Total de ejemplos de entrenamiento
klass
neutral     1485
positive     968
negative     689
Name: count, dtype: int64
Datos train: (3142, 300)
Etiquetas train: (3142,)
Datos test: (786, 300)
Etiquetas test: (786,)


# Codificar las etiquetas

In [2]:
# TODO: Codificar las etiquetas de los datos a una forma categórica numérica: LabelEncoder.

le = LabelEncoder()
# Normalizar las etiquetas a una codificación ordinal para entrada del clasificador
Y_encoded= le.fit_transform(Y)
print("Clases:")
print(le.classes_)
print("Clases codificadas:")
print(le.transform(le.classes_))


Clases:
['negative' 'neutral' 'positive']
Clases codificadas:
[0 1 2]


# Preparar los conjuntos de datos  para el entrenamiento, validación y prueba

In [3]:

# Dividir el conjunto de entrenamiento en:  entrenamiento (80%) y validación (10%)
X_train, X_val, Y_train, Y_val =  train_test_split(X, Y_encoded, test_size=0.1, stratify=Y_encoded, random_state=42)



# Codificar las clases en forma one-hot 
NUM_CLASSES = 3
Y_train_one_hot = nn.functional.one_hot(torch.from_numpy(Y_train), num_classes=NUM_CLASSES).float()


# Crear minibatches en PyTorch usando DataLoader
def create_minibatches(X, Y, batch_size):
    # Recibe los documentos en X y las etiquetas en Y
    dataset = TensorDataset(X, Y) # Cargar los datos en un dataset de tensores
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    # loader = DataLoader(dataset, batch_size=batch_size)
    return loader


In [4]:
X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape

((2827, 300), (2827,), (315, 300), (315,), (786, 300), (786,))

In [5]:
Y_train_one_hot[:5],  Y_train_one_hot.shape

(tensor([[0., 1., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]),
 torch.Size([2827, 3]))

# Definición de la arquitectura de la red

In [6]:

# Definir la red neuronal en PyTorch heredando de la clase base de Redes Neuronales: Module
class RedNeuronal(nn.Module):
    def __init__(self, tam_entrada, tam_salida):
        super().__init__()
        # Definición de capas, funciones de activación e inicialización de pesos
        tam_entrada_capa_oculta_1 = 128
        tam_entrada_capa_oculta_2 = 8 
        self.fc1 = nn.Linear(tam_entrada, tam_entrada_capa_oculta_1)
        self.fc2 = nn.Linear(tam_entrada_capa_oculta_1, tam_entrada_capa_oculta_2)
        self.salida = nn.Linear(tam_entrada_capa_oculta_2, tam_salida)
        self.activacion= nn.PReLU()

        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.salida.weight)

        if self.fc1.bias is not None:
            nn.init.zeros_(self.fc1.bias)
        if self.fc2.bias is not None:
            nn.init.zeros_(self.fc2.bias)        
        if self.salida.bias is not None:
            nn.init.zeros_(self.salida.bias)        

    
    def forward(self, X):
        # Definición del orden de conexión de las capas y aplición de las funciones de activación
        x = self.fc1(X)
        x = self.activacion(x)
        x = self.fc2(x)
        x = self.activacion(x)
        x = self.salida(x)
        return x

# Entrenamiento de la red

In [8]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Establecer los parámetros de la red

# Parámetros de la red
tam_entrada =  X_train.shape[1]

tam_salida = 3   # 2 clases

epochs = 30 # variar el número de épocas, para probar que funciona la programación 
                 # solo usar 2 épocas, para entrenamiento total usar por ejemplo 1000 épocas
learning_rate = 0.001 # Generalmente se usan learning rate pequeños (0.001), 

# Se recomiendan tamaños de batch_size potencias de 2: 16, 32, 64, 128, 256
# Entre mayor el número más cantidad de memoria se requiere para el procesamiento
batch_size = 128 # definir el tamaño del lote de procesamiento 


# Convertir los datos de entrenamiento y etiquetas a tensores  de PyTorch

X_train_t = torch.from_numpy(X_train)
X_train_t = X_train_t.to(torch.float32)
Y_train_t = Y_train_one_hot

X_val_t = torch.from_numpy(X_val)
X_val_t = X_val_t.to(torch.float32)


# Crear la red
modelo_red_neuronal = RedNeuronal(tam_entrada, tam_salida)

# Definir la función de pérdida
# Entropía Cruzada 
criterion = nn.CrossEntropyLoss() 

# Definir el optimizador
#Parámetros del optimizador: parámetros del modelo y learning rate 
# Adaptive Moment Estimation
optimizer = optim.Adam(modelo_red_neuronal.parameters(), lr=learning_rate)

# Entrenamiento
print("Iniciando entrenamiento en PyTorch")


for epoch in range(epochs):
# Poner el modelo en modo de entrenamiento
    modelo_red_neuronal.train()  
    lossTotal = 0
    #definir el batch_size
    dataloader = create_minibatches(X_train_t, Y_train_t, batch_size=batch_size)
    for X_tr, y_tr in dataloader:
        # inicializar los gradientes en cero para cada época
        optimizer.zero_grad()
        
        # Propagación hacia adelante
        y_pred = modelo_red_neuronal(X_tr)  #invoca al método forward de la clase MLP
        # Calcular el error MSE
        loss = criterion(y_pred, y_tr)
        #Acumular el error 
        lossTotal += loss.item()
        
        # Propagación hacia atrás: cálculo de los gradientes de los pesos y bias
        loss.backward()
        
        # actualización de los pesos: regla de actualización basado en el gradiente:
        #  
        optimizer.step()
        if np.random.random() < 0.1:
            print(f"Batch Error : {loss.item()}")

    print(f"Época {epoch+1}/{epochs}, Pérdida: {lossTotal/len(dataloader)}")
    
    # Evalúa el modelo con el conjunto de validación
    modelo_red_neuronal.eval()  # Establecer el modo del modelo a "evaluación"
    with torch.no_grad():  # No  calcular gradientes 
        y_pred = modelo_red_neuronal(X_val_t)
        # Obtiene una única clase, la más probable
        y_pred = torch.argmax(y_pred, dim=1)
        print(f"Época {epoch+1}/{epochs}")
        print("P=", precision_score(Y_val, y_pred, average='macro'))
        print("R=", recall_score(Y_val, y_pred, average='macro'))
        print("F1=", f1_score(Y_val, y_pred, average='macro'))
        print("Acc=", accuracy_score(Y_val, y_pred))


Iniciando entrenamiento en PyTorch
Batch Error : 1.0693750381469727
Batch Error : 1.0727605819702148
Época 1/30, Pérdida: 1.0640511512756348
Época 1/30
P= 0.15767195767195766
R= 0.3333333333333333
F1= 0.21408045977011494
Acc= 0.473015873015873
Batch Error : 1.0078034400939941
Época 2/30, Pérdida: 1.029496527236441
Época 2/30
P= 0.4394939493949395
R= 0.36322332157106946
F1= 0.27797623880273875
Acc= 0.4984126984126984
Época 3/30, Pérdida: 0.9963932762975278
Época 3/30
P= 0.44830508474576275
R= 0.3850411679236145
F1= 0.3160853160853161
Acc= 0.5174603174603175
Batch Error : 0.9965870380401611
Época 4/30, Pérdida: 0.9464147116826929
Época 4/30
P= 0.41694537346711263
R= 0.41285546253373
F1= 0.36176470588235293
Acc= 0.5365079365079365
Batch Error : 0.9010958671569824
Época 5/30, Pérdida: 0.9259017990983051
Época 5/30
P= 0.4058333333333333
R= 0.46384833598560854
F1= 0.4187541101213607
Acc= 0.5714285714285714
Batch Error : 0.8826020956039429
Batch Error : 0.9293391108512878
Época 6/30, Pérdida:

/Users/admin/opt/miniconda3/envs/CE/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/admin/opt/miniconda3/envs/CE/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/admin/opt/miniconda3/envs/CE/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

Batch Error : 0.7828763723373413
Batch Error : 1.1228022575378418
Época 15/30, Pérdida: 0.7720405744469684
Época 15/30
P= 0.6148519061318279
R= 0.5787508469063977
F1= 0.5880550949155289
Acc= 0.6285714285714286
Batch Error : 0.7323153018951416
Batch Error : 0.7100321054458618
Batch Error : 0.6632977724075317
Época 16/30, Pérdida: 0.7510512315708658
Época 16/30
P= 0.6107541382410493
R= 0.5556341043482271
F1= 0.5645468998410176
Acc= 0.6158730158730159
Batch Error : 0.7486873269081116
Batch Error : 0.8050156235694885
Época 17/30, Pérdida: 0.7393786725790604
Época 17/30
P= 0.6466487497556122
R= 0.5523270330516606
F1= 0.560933238583887
Acc= 0.6222222222222222
Batch Error : 0.804322361946106
Batch Error : 0.7041562795639038
Época 18/30, Pérdida: 0.7265563762706259
Época 18/30
P= 0.6309758339900184
R= 0.5690231638718338
F1= 0.5761512890345387
Acc= 0.6317460317460317
Batch Error : 0.7128019332885742
Batch Error : 0.5991825461387634
Época 19/30, Pérdida: 0.7236901132956796
Época 19/30
P= 0.59825

### Modo para predicción de datos

In [9]:
# TODO: Transformar el dataset de test con los mismos preprocesamientos y al  espacio de 
# representación vectorial que el modelo entrenado, es decir, al espacio de la matriz TFIDF

# Convertir los datos de prueba a tensores de PyTorch

X_t = torch.from_numpy(X_test)
X_t = X_t.to(torch.float32)

# Desactivar el comportamiento de modo de  entrenamiento: por ejemplo, capas como Dropout
modelo_red_neuronal.eval()  # Establecer el modo del modelo a "evaluación"

with torch.no_grad():  # No  calcular gradientes 
    y_pred_test= modelo_red_neuronal(X_t)

# y_test_pred contiene las predicciones

# Obtener la clase real
y_pred_test = torch.argmax(y_pred_test, dim=1)

print(y_pred_test)


tensor([1, 1, 0, 1, 1, 1, 2, 2, 1, 0, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 0, 1, 2, 1,
        1, 0, 2, 1, 0, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 2, 0, 1, 2, 2, 1, 1, 2, 1,
        1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 0, 1, 2, 1, 2, 1, 0, 1, 1, 2, 0, 1,
        2, 1, 1, 1, 1, 2, 2, 0, 1, 2, 0, 1, 1, 1, 0, 2, 1, 1, 2, 0, 1, 1, 2, 1,
        1, 1, 1, 1, 0, 1, 1, 2, 1, 0, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2,
        2, 1, 1, 0, 0, 1, 0, 1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2,
        2, 2, 1, 1, 0, 2, 0, 0, 1, 1, 2, 0, 2, 0, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1,
        1, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 0, 2, 1, 0, 1, 1, 0, 2, 2, 1,
        2, 2, 1, 1, 1, 2, 1, 2, 1, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 1, 0, 0, 1, 2, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 2, 1, 1, 1, 2, 2, 1,
        1, 2, 1, 2, 1, 0, 1, 1, 2, 1, 0, 1, 1, 2, 0, 1, 1, 2, 2, 2, 2, 2, 2, 0,
        1, 0, 1, 1, 0, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 0, 2, 0, 2, 2,
        1, 0, 2, 1, 2, 1, 1, 1, 0, 1, 2,

### Evaluación

In [10]:
# TODO: Evaluar el modelo con las predicciones obtenidas y las etiquetas esperadas: 
# classification_report y  matriz de confusión (métricas Precisión, Recall, F1-measaure, Accuracy)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
Y_t = le.transform(Y_test)

print(confusion_matrix(Y_t, y_pred_test))
print(classification_report(Y_t, y_pred_test, digits=4, zero_division='warn'))


[[ 75  60  38]
 [ 34 287  50]
 [ 27  72 143]]
              precision    recall  f1-score   support

           0     0.5515    0.4335    0.4854       173
           1     0.6850    0.7736    0.7266       371
           2     0.6190    0.5909    0.6047       242

    accuracy                         0.6425       786
   macro avg     0.6185    0.5993    0.6056       786
weighted avg     0.6353    0.6425    0.6360       786



# Guardar el modelo

In [11]:

# Obtener el state_dict (diccionario de pesos y sesgos)
state_dict = modelo_red_neuronal.state_dict()

# Guardar el state_dict
torch.save(state_dict, "./data/red_neuronal_parametros.pth")
